In [1]:
# import gym
from header import *
from utils import *
from replay_buffer import *
from models import poly_net, val_net
from reconstructors import sigpy_solver
from policies import DQN
from importlib import reload
memory_len = 10
t_backtrack = 3
heg = 192
wid = 144

<frozen importlib._bootstrap>:228: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 88 from PyObject
/opt/anaconda/envs/pyenv/lib/python3.9/site-packages/h5py/__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters


In [2]:
datapath = '/mnt/shared_a/OCMR/OCMR_fully_sampled_images/'
ncfiles = list([])
for file in os.listdir(datapath):
    if file.endswith(".pt"):
        ncfiles.append(file)
loader = ocmrLoader(ncfiles)

In [3]:
class AC1_trainer():
    def __init__(self, dataloader, polynet, valnet,
                  fulldim:int=144,base:int=10,budget:int=50,
                  gamma:float=.8,
                  horizon:int=None,
                  max_trajectories:int=100,
                  lr:float=.1,
                  init_base:int=10,
                  L:float=5e-3,
                  max_iter:int=100,
                  solver:str='ADMM',
                  device=torch.device('cpu'),
                  save_dir:str='/home/huangz78/rl_samp/'):
        self.dataloader = dataloader
        self.dataloader.reset()
        self.polynet    = polynet
        self.valnet     = valnet
        self.fulldim    = fulldim
        self.base       = int(base)
        self.budget     = int(budget)
        self.gamma      = gamma
        if horizon is None:
            self.horizon = self.budget
        else:
            self.horizon = int(horizon)
        self.lr         = lr
        self.optimizer_val  = optim.Adam(self.valnet.parameters(),  lr=lr)
        self.optimizer_poly = optim.Adam(self.polynet.parameters(), lr=lr)
        self.init_base  = int(init_base)
        self.max_trajectories = int(max_trajectories)
        ## reconstructor parameters
        self.L = L
        self.max_iter = int(max_iter)
        self.solver = 'ADMM'
        self.device = device
        
        self.reward_per_horizon = []
        self.save_dir = save_dir
    
    def get_action(self, curr_obs, mask=None):
        '''
        here the mask is 1D
        '''
        if mask is not None:
            assert(len(mask.shape)==1)
        res  = self.polynet(curr_obs,mask) # Jul 5, add mask here as second input
        loc  = torch.argmax(res)
        prob = res.gather(dim=1,index=loc.long().view(-1,1)).squeeze()
        return loc, prob
        
    def step(self, action, target_gt, mask):
        '''
        action: [1] TODO: adding multiple lines at a time
        mask:[W]
        data:[N1HW]
        target:[N1HW]
        '''
        ### observe target_gt with old freq info
        target_obs_freq = fft_observe(target_gt,mask,return_opt='freq',roll=True) # roll=True because using sigpy
        img_recon = sigpy_solver(target_obs_freq, 
                                 L=self.L,max_iter=self.max_iter,solver=self.solver,
                                 heg=target_gt.shape[2],wid=target_gt.shape[3])
        old_nrmse = NRMSE(img_recon,target_gt)
        
        ### observe target_gt with new freq info
        mask[action] = 1 # incorporate action into mask
        target_obs_freq = fft_observe(target_gt,mask,return_opt='freq',roll=True) # roll=True because using sigpy
        next_obs  = sigpy_solver(target_obs_freq, 
                                 L=self.L,max_iter=self.max_iter,solver=self.solver,
                                 heg=target_gt.shape[2],wid=target_gt.shape[3])
        new_nrmse = NRMSE(next_obs,target_gt)
        reward    = old_nrmse - new_nrmse
        
        return next_obs, reward
        
    def run(self):
        for trajectory in range(self.max_trajectories):
            
            print(f'trajectory [{trajectory+1}/{self.max_trajectories}]')
            mask = mask_naiveRand(self.fulldim,fix=self.base,other=0,roll=False) # curr_state
            I = 1
            reward_horizon = 0
            for t in range(self.horizon):
                breakpoint()
                data_source, data_target = self.dataloader.load()
                curr_obs = fft_observe(data_source, mask)
                action, prob = self.get_action(curr_obs, mask=mask)
                
                next_obs_last_slice, reward = self.step(action, data_target, copy.deepcopy(mask)) 
                reward_horizon += reward
                v = self.valnet(curr_obs)
                with torch.no_grad():
                    next_obs = torch.concat((curr_obs[:,1:,:,:],next_obs_last_slice),dim=1)
                    vnew  = self.valnet(next_obs)
                    delta = reward + self.gamma * vnew  - v # should check if delta == 0
                self.optimizer_val.zero_grad()
                val_loss = - delta * v
                val_loss.backward()
                self.optimizer_val.step()
                
                self.optimizer_poly.zero_grad()
                poly_loss = - I * delta * torch.log(prob)
                poly_loss.backward()
                self.optimizer_poly.step()
                
                I *= self.gamma
                mask[action] = 1
                print(f'step: {self.steps}, poly_loss: {poly_loss.detach().item():.4f}, val_loss: {val_loss.detach().item():.4f}, reward: {reward.mean().item():.4f}, \n mask sum: {mask.sum().item()}')
                torch.cuda.empty_cache()
            self.reward_per_horizon.append(reward_horizon)
    
    def save(self):
        filename = f'AC1_hist.pt'
        torch.save(
                    {
                        "polynet_weights": self.polynet.state_dict(),
                        "valnet_weights": self.valnet.state_dict(),
                        "reward_per_horizon":self.reward_per_horizon,
                    },
                    self.save_dir + filename,
                )

In [4]:
p_net   = poly_net(samp_dim=wid,softmax=True)
v_net   = val_net()
trainer = AC1_trainer(loader, polynet=p_net, valnet=v_net)
trainer.run()

current file: fs_0019_3T.pt
trajectory [1/100]
> /tmp/ipykernel_21748/2443126542.py(85)run()
     83             for t in range(self.horizon):
     84                 breakpoint()
---> 85                 data_source, data_target = self.dataloader.load()
     86                 curr_obs = fft_observe(data_source, mask)
     87                 action, prob = self.get_action(curr_obs, mask=mask)

ipdb> n
> /tmp/ipykernel_21748/2443126542.py(86)run()
     84                 breakpoint()
     85                 data_source, data_target = self.dataloader.load()
---> 86                 curr_obs = fft_observe(data_source, mask)
     87                 action, prob = self.get_action(curr_obs, mask=mask)
     88 

ipdb> n
> /tmp/ipykernel_21748/2443126542.py(87)run()
     85                 data_source, data_target = self.dataloader.load()
     86                 curr_obs = fft_observe(data_source, mask)
---> 87                 action, prob = self.get_action(curr_obs, mask=mask)
     88 
     89

BdbQuit: 

In [ ]:
class RL_trainer():
    def __init__(self,dataloader,policy,memory,episodes:int=10,eps:float=1e-3,
                 fulldim:int=144,base:int=10,budget:int=50,
                 save_dir:str='/home/huangz78/rl_samp/'):
        self.dataloader = dataloader
        self.dataloader.reset()
        
        self.policy   = policy
        self.memory   = memory
        self.episodes = episodes
        self.epi      = 0
        self.fulldim  = fulldim
        self.base     = base
        self.budget   = budget
        self.eps      = eps
        self.training_record = {'loss':[],'grad_norm':[],'q_values_mean':[],'q_values_std':[]}
        self.steps    = 0
        self.save_dir = save_dir
    def train(self):      
        # run training
        while self.epi < self.episodes:
            print(f'episode [{self.epi+1}/{self.episodes}]')
            mask = mask_naiveRand(self.fulldim,fix=self.base,other=0,roll=False)   
            # one mask at a time, start with a low frequency mask
            while mask.sum() < self.budget + self.base:
                self.steps += 1
#                 print(f'step: {self.steps}, beginning, mask sum: {mask.sum().item()}')
                data_source, data_target = self.dataloader.load()
                mask_RL   = copy.deepcopy(mask)
                mask_rand = copy.deepcopy(mask)
#                 epsilon = _get_epsilon(steps_epsilon, self.options)
                curr_obs = fft_observe(data_source,mask_RL)
                action   = self.policy.get_action(curr_obs, mask=mask_RL, eps_threshold=self.eps)
                next_obs, reward = self.policy.step(action, data_target, mask_RL)
#                 print(f'step: {self.steps}, policy.step, mask_RL sum: {mask_RL.sum().item()}')
                
                self.memory.push(curr_obs, mask, action, next_obs, reward)
                mask = copy.deepcopy(mask_RL)
#                 print(f'step: {self.steps}, assign, mask sum: {mask.sum().item()}')
                
                ### compare with random policy
                with torch.no_grad():
                    action_rand = self.policy.get_rand_action(mask=mask_rand)
                    _, reward_rand = self.policy.step(action_rand, data_target, mask_rand)
                ###
                
                update_results = self.policy.update_parameters()
                if update_results is not None:
                    for key in self.training_record.keys():
                        self.training_record[key].append(update_results[key])
                    curr_loss = update_results['loss']
                    print(f'step: {self.steps}, loss: {curr_loss:.4f}, RL reward: {reward.mean().item():.4f}, Rand reward: {reward_rand.mean().item():.4f} \n mask sum: {mask.sum().item()}')
                    torch.cuda.empty_cache()
                else:
                    print(f'step: {self.steps}, burn in, mask sum: {mask.sum().item()}')
                
                if type(self.policy).__name__.lower() == 'ddqn':
                    if self.steps % self.policy.freq_dqn_checkpoint_save == 0:
                        self.save()
                    if self.steps % self.policy.target_net_update_freq == 0:
                        self.target_net.load_state_dict(self.policy.model.state_dict())
            self.dataloader.reset()
            self.epi += 1
    
    def save(self):
        filename = f'{type(self.policy).__name__}_hist.pt'
        if type(self.policy).__name__.lower() == 'ddqn': 
            torch.save(
                    {
                        "dqn_weights": self.policy.model.state_dict(),
                        "target_weights": self.policy.target_net.state_dict(),
                        "steps": self.steps,
                        "training_record":self.training_record,
                    },
                    self.save_dir + filename,
                )
        elif type(self.policy).__name__.lower() == 'dqn':
            torch.save(
                    {
                        "dqn_weights": self.policy.model.state_dict(),
                        "steps": self.steps,
                        "training_record":self.training_record,
                    },
                    self.save_dir + filename,
                )

In [ ]:
memory  = ReplayMemory(capacity=memory_len,
                       curr_obs_shape=(t_backtrack,heg,wid),
                       mask_shape=(wid),
                       next_obs_shape=(1,heg,wid),
                       batch_size=2,
                       burn_in=2)
model   = poly_net(samp_dim=wid)
policy  = DQN(model,memory)
trainer = RL_trainer(loader,policy,memory)

In [ ]:
import dqn
reload(dqn)
from dqn import DQN

import replay_buffer
reload(replay_buffer)
from replay_buffer import *

import utils
reload(utils)
from utils import *

In [ ]:
trainer.train()

In [ ]:
class RL_tester():
    def __init__(self,dataloader,policy,
                 fulldim:int=144,base:int=10,budget:int=50):
        self.dataloader = dataloader
        self.dataloader.reset()
        
        self.policy   = policy
        self.epi = 0
        self.episodes = self.dataloader.files
        self.fulldim = fulldim # full size of the dimension to be sampled
        self.base = base
        self.budget = budget
        self.eps = eps
        self.test_record = {'loss':[],'grad_norm':[],'q_values_mean':[],'q_values_std':[]}
        self.testRec = np.zeros((self.episodes))
        self.steps = 0
    
    def run(self):  
        self.policy.model.eval()
        # run training
        while self.epi < self.episodes:
            print(f'file [{self.epi+1}/{self.episodes}]')
            mask = mask_naiveRand(self.fulldim,fix=self.base,other=0,roll=False)
            epi_loss = 0
            slice_count = 0
            # one mask at a time, start with a low frequency mask
            while mask.sum() < self.budget + self.base:
                self.steps += 1
                data_source, data_target = self.dataloader.load()
#                 epsilon = _get_epsilon(steps_epsilon, self.options)
                curr_obs = fft_observe(data_source,mask)
                action   = self.policy.get_action(curr_obs, mask=mask, eps_threshold=self.eps)
                next_obs, _ = self.policy.step(action, data_target, mask)  
                
#                 ### compare with random policy
#                 with torch.no_grad():
#                     action_rand = self.policy.get_rand_action(mask=mask_rand)
#                     _, reward_rand = self.policy.step(action_rand, data_target, mask_rand)
#                 ###
            # 
            self.dataloader.reset_iter()
            self.dataloader.train_mode = False
            runFlag = True
            while runFlag:
                data_source, data_target = self.dataloader.load()
                curr_obs  = fft_observe(data_source,mask,return_opt='freq')
                img_recon = sigpy_solver(curr_obs, 
                                 L=self.policy.L,max_iter=self.policy.max_iter,solver=self.policy.solver,
                                 heg=curr_obs.shape[2],wid=curr_obs.shape[3])
                curr_nrmse = NRMSE(img_recon,data_source)
                epi_loss += curr_nrmse * curr_obs.shape[0]
                slice_count += curr_obs.shape[0]
                if data_source is None:
                    runFlag = False
            self.testRec[self.epi] = epi_loss / slice_count
#                 if self.steps % self.options.target_net_update_freq == 0:
#                     self.logger.info("Updating target network.")
#                     self.target_net.load_state_dict(self.policy.state_dict())
            self.dataloader.reset()
            self.epi += 1
        return np.mean(self.testRec)